# Interactive Data Visualization with Bokeh

<a href="https://colab.research.google.com/github/bradleyboehmke/uc-bana-4080/blob/main/example-notebooks/14_data_viz_bokeh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook accompanies [this textbook chapter](https://bradleyboehmke.github.io/uc-bana-4080/15-data-viz-bokeh.html) and allows you to run the code examples interactively.

## Prerequisites

In [ ]:
import pandas as pd

# Our main plotting package (must have explicit import of submodules)
import bokeh.io
import bokeh.models
import bokeh.plotting
import bokeh.transform

# Enable viewing Bokeh plots in the notebook
bokeh.io.output_notebook()

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/bradleyboehmke/uc-bana-4080/refs/heads/main/data/ames_clean.csv')
df.head()

## Bokeh's grammar and our first plot with Bokeh

In [ ]:
# Create the figure, stored in variable `p`
p = bokeh.plotting.figure(
    frame_width=700,
    frame_height=350,
    title='Relationship between home sale price and living area \nAmes, Iowa (2006-2010)',
    x_axis_label='Living Area (Square feet)',
    y_axis_label='Sale Price'
)

In [ ]:
source = bokeh.models.ColumnDataSource(df)

In [ ]:
p.scatter(
    source=source,
    x='GrLivArea',
    y='SalePrice',
    alpha=0.25
);

In [ ]:
p.yaxis.formatter = bokeh.models.NumeralTickFormatter(format="$,")
p.xaxis.formatter = bokeh.models.NumeralTickFormatter(format=",")

tooltips = [("Sale Price","@SalePrice"),("SqFt","@GrLivArea")]
hover = bokeh.models.HoverTool(tooltips=tooltips, mode='mouse')
p.add_tools(hover)

In [ ]:
bokeh.io.show(p)

## Coloring with other dimensions

In [ ]:
# Create the figure, stored in variable `p`
p = bokeh.plotting.figure(
    frame_width=700,
    frame_height=350,
    title='Relationship between home sale price and living area \nAmes, Iowa (2006-2010)',
    x_axis_label='Living Area (Square feet)',
    y_axis_label='Sale Price'
)

source = bokeh.models.ColumnDataSource(df)

# create color mapper
color_mapper = bokeh.transform.factor_cmap(
    'CentralAir',
    palette=['red', 'blue'],
    factors=df['CentralAir'].unique()
    )

p.scatter(
    source=source,
    x='GrLivArea',
    y='SalePrice',
    marker='circle',
    alpha=0.25,
    fill_color=color_mapper,
    line_color=color_mapper,
    legend_field='CentralAir'
)

p.legend.title = "Has central air"

p.yaxis.formatter = bokeh.models.NumeralTickFormatter(format="$,")
p.xaxis.formatter = bokeh.models.NumeralTickFormatter(format=",")

tooltips = [("Sale Price","@SalePrice"),("SqFt","@GrLivArea")]
hover = bokeh.models.HoverTool(tooltips=tooltips, mode='mouse')
p.add_tools(hover)

bokeh.io.show(p)

## Saving Bokeh plots

In [ ]:
bokeh.io.save(
    p,
    filename='ames_sale_price_vs_living_area.html',
    title='Bokeh plot'
);

## Exercise: Exploring Housing Trends with Bokeh

In this exercise set, you’ll revisit the [Ames Housing dataset](https://github.com/bradleyboehmke/uc-bana-4080/blob/main/data/ames_clean.csv) and apply what you've learned about **Bokeh** to build interactive plots from scratch. These visualizations will give you hands-on practice with building charts, customizing tooltips, applying categorical coloring, and creating simple layouts.

Use the [Ames Housing dataset](https://github.com/bradleyboehmke/uc-bana-4080/blob/main/data/ames_clean.csv) we used in this chapter, and import the necessary Bokeh modules as needed:

```python
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.transform import factor_cmap
from bokeh.layouts import row, column
from bokeh.models import HoverTool

output_notebook()

# can also use this URL to download the dataset directly
# https://raw.githubusercontent.com/bradleyboehmke/uc-bana-4080/refs/heads/main/data/ames_clean.csv
df = pd.read_csv("data/ames_clean.csv")  # adjust path as needed
df.head()